In [69]:
import requests
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [82]:
browser=webdriver.Chrome()

#load the website link
browser.get('https://www.icc-cricket.com/tournaments/t20cricketworldcup/matches')

#maximizing the window
browser.maximize_window()
time.sleep(6)

#accepting the cookies
browser.find_element(By.ID,'onetrust-accept-btn-handler').click()
time.sleep(2)

#scroll down
#browser.execute_script("window.scrollBy(0, 1000)")
match_date, match_title, match_place, match_weather = [], [], [], []
match_team_a, match_team_b, match_team_a_score, match_team_b_score = [], [], [], []
match_player_of_match, match_player_of_match_score, match_toss_winner = [], [], []
match_team_win=[]


match_center_div = browser.find_elements(By.CSS_SELECTOR, "div.si-card-footer")
print(len(match_center_div))

for i in tqdm(range(len(match_center_div))):
    links = match_center_div[i].find_elements(By.TAG_NAME, "a")
    
    if len(links) > 1:
        links[1].click()
        
        # Wait for match details to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='si-title si-meta']"))
        )
        
        # Gather match information
        match_title.append(browser.find_element(By.XPATH, "//span[@class='si-title si-meta']").text)
        match_date.append(browser.find_element(By.XPATH, "//span[@class='si-match-time si-meta']").text)
        match_place.append(browser.find_element(By.XPATH, "//span[@class='si-venue si-meta']").text)
        match_weather.append(browser.find_element(By.XPATH, "//span[@class='si-weather si-meta']").text)
        match_team_a.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-team-name']").text)
        match_team_b.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-team-name']").text)
        match_team_a_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-score']").text)
        match_team_b_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-score']").text)
        match_team_win.append(browser.find_element(By.XPATH,"//div[@class='si-team-status']").text)
        # Player of the Match
        player_of_match_fname = browser.find_element(By.XPATH, "//div[@class='si-player-info']//span[@class='si-fname si-name']").text
        player_of_match_lname = browser.find_element(By.XPATH, "//div[@class='si-player-info']//span[@class='si-lname si-name']").text
        match_player_of_match.append(f"{player_of_match_fname} {player_of_match_lname}")
        match_player_of_match_score.append(browser.find_element(By.XPATH, "//div[@class='si-player-allrounder']").text)

        # Click on the 5th tab (Match Details)
        fifth_tab = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "ul[role='tablist'] li:nth-of-type(5) button"))
        )
        browser.execute_script("arguments[0].scrollIntoView();", fifth_tab)
        browser.execute_script("arguments[0].click();", fifth_tab)
        
        # Wait for toss winner to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "(//div[@class='si-matchinfo-item'])[5]"))
        )
        match_toss_winner.append(browser.find_element(By.XPATH, "(//div[@class='si-matchinfo-item'])[5]").text)

        # Navigate back to the match center
        browser.back()

        # Wait for match center to reload
        WebDriverWait(browser, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.si-card-footer"))
        )
browser.quit()
 

55


100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [02:19<00:00,  2.53s/it]


In [89]:
data={'DATE':match_date,
     'TITLE':match_title,
     'VENUE':match_place,
     #'WHEATHER CONDITION':match_wheather,
      'TOSS WINNER':match_toss_winner,
     'TEAM_A':match_team_a,
     'TEAM_A_SCORE':match_team_a_score,
     'TEAM_B':match_team_b,
     'TEAM_B_SCORE':match_team_b_score,
      'TEAM WON':match_team_win,
     'PLAYER OF THE MATCH':match_player_of_match,
     'PLAYER OF THE MATCH SCORE':match_player_of_match_score}
df=pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   DATE                       51 non-null     object
 1   TITLE                      51 non-null     object
 2   VENUE                      51 non-null     object
 3   TOSS WINNER                51 non-null     object
 4   TEAM_A                     51 non-null     object
 5   TEAM_A_SCORE               51 non-null     object
 6   TEAM_B                     51 non-null     object
 7   TEAM_B_SCORE               51 non-null     object
 8   TEAM WON                   51 non-null     object
 9   PLAYER OF THE MATCH        51 non-null     object
 10  PLAYER OF THE MATCH SCORE  51 non-null     object
dtypes: object(11)
memory usage: 4.5+ KB


In [90]:
df.head()

,DATE,TITLE,VENUE,TOSS WINNER,TEAM_A,TEAM_A_SCORE,TEAM_B,TEAM_B_SCORE,TEAM WON,PLAYER OF THE MATCH,PLAYER OF THE MATCH SCORE
0,Saturday 29th June 2024,"ICC Men's T20 World Cup, 2024 - Final","Kensington Oval, Bridgetown, Barbados",TOSS\nIndia elected to bat,SOUTH AFRICA,169/8 (20.0),INDIA,176/7 (20.0),India beat South Africa by 7 runs,Virat KOHLI,76\n(59)
1,Thursday 27th June 2024,"ICC Men's T20 World Cup, 2024 - 2nd Semi-Final","Guyana National Stadium, Guyana",TOSS\nEngland elected to field,INDIA,171/7 (20.0),ENGLAND,103 (16.4),India beat England by 68 runs,Axar PATEL,10\n(6)\n&\n3/23\n(4)
2,Thursday 27th June 2024,"ICC Men's T20 World Cup, 2024 - 1st Semi-Final","Brian Lara Cricket Academy, Tarouba, Trinidad ...",TOSS\nAfghanistan elected to bat,SOUTH AFRICA,60/1 (8.5),AFGHANISTAN,56 (11.5),South Africa beat Afghanistan by 9 wickets,Marco JANSEN,3/16\n(3)
3,Tuesday 25th June 2024,"ICC Men's T20 World Cup, 2024 - Super Eight - ...","Arnos Vale Ground, Arnos Vale, St Vincent",TOSS\nAfghanistan elected to bat,AFGHANISTAN,115/5 (20.0),BANGLADESH,105 (17.5),Afghanistan beat Bangladesh by 8 runs (DLS met...,Naveen-ul-Haq,4/26\n(3.5)
4,Monday 24th June 2024,"ICC Men's T20 World Cup, 2024 - Super Eight - ...","Sir Vivian Richards Stadium, North Sound, Antigua",TOSS\nSouth Africa elected to field,WEST INDIES,135/8 (20.0),SOUTH AFRICA,124/7 (16.1),South Africa beat West Indies by 3 wickets (DL...,Tabraiz SHAMSI,3/27\n(4)


In [91]:
df.to_csv('D:\WEB SCRAPING\projects\icc 2024 t20 world cup fixtures-2.csv',index=False)

In [ ]:
match_center_div = browser.find_elements(By.CSS_SELECTOR, "div.si-card-footer")
print(len(match_center_div))

for i in tqdm(range(len(match_center_div))):
    links = match_center_div[i].find_elements(By.TAG_NAME, "a")
    
    if len(links) > 1:
        links[1].click()
        
        # Wait for match details to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='si-title si-meta']"))
        )
        
        # Gather match information
        match_title.append(browser.find_element(By.XPATH, "//span[@class='si-title si-meta']").text)
        match_date.append(browser.find_element(By.XPATH, "//span[@class='si-match-time si-meta']").text)
        match_place.append(browser.find_element(By.XPATH, "//span[@class='si-venue si-meta']").text)
        match_weather.append(browser.find_element(By.XPATH, "//span[@class='si-weather si-meta']").text)
        match_team_a.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-team-name']").text)
        match_team_b.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-team-name']").text)
        match_team_a_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-score']").text)
        match_team_b_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-score']").text)

        # Player of the Match
        player_of_match_fname = browser.find_element(By.XPATH, "//div[@class='si-player-info']//span[@class='si-fname si-name']").text
        player_of_match_lname = browser.find_element(By.XPATH, "//div[@class='si-player-info']//span[@class='si-lname si-name']").text
        match_player_of_match.append(f"{player_of_match_fname} {player_of_match_lname}")
        match_player_of_match_score.append(browser.find_element(By.XPATH, "//div[@class='si-player-allrounder']").text)

        # Click on the 5th tab (Match Details)
        fifth_tab = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "ul[role='tablist'] li:nth-of-type(5) button"))
        )
        browser.execute_script("arguments[0].scrollIntoView();", fifth_tab)
        fifth_tab.click()
        
        # Wait for toss winner to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "(//div[@class='si-matchinfo-item'])[5]"))
        )
        match_toss_winner.append(browser.find_element(By.XPATH, "(//div[@class='si-matchinfo-item'])[5]").text)

        # Navigate back to the match center
        browser.back()

        # Wait for match center to reload
        WebDriverWait(browser, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.si-card-footer"))
        )
browser.quit()
 

In [ ]:
match_date,match_title,match_place,match_wheather,match_team_a,match_team_b,match_team_a_score,match_team_b_score=[],[],[],[],[],[],[],[]
match_player_of_match,match_player_of_match_score,match_toss_winner=[],[],[]
#Match Center
match_center_div= browser.find_elements(By.CSS_SELECTOR, "div.si-card-footer")
print(len(match_center_div))
for i in tqdm(range(len(match_center_div))):
    links = match_center_div[i].find_elements(By.TAG_NAME, "a")
    if len(links) > 1:
        links[1].click()
        time.sleep(3)
        match_title.append(browser.find_element(By.XPATH, "//span[@class='si-title si-meta']").text)
        match_date.append(browser.find_element(By.XPATH, "//span[@class='si-match-time si-meta']").text)
        match_place.append(browser.find_element(By.XPATH, "//span[@class='si-venue si-meta']").text)
        match_wheather.append(browser.find_element(By.XPATH, "//span[@class='si-weather si-meta']").text)
        match_team_a.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-team-name']").text)
        match_team_b.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-team-name']").text)
        match_team_a_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-a']//span[@class='si-score']").text)
        match_team_b_score.append(browser.find_element(By.XPATH,"//div[@class='si-team si-team-b']//span[@class='si-score']").text)
        match_player_of_match.append(browser.find_element(By.XPATH,"//div[@class='si-player-info']//span[@class='si-fname si-name']").text + browser.find_element(By.XPATH,f"//div[@class='si-player-info']//span[@class='si-lname si-name']").text)
        match_player_of_match_score.append(browser.find_element(By.XPATH,"//div[@class='si-player-allrounder']").text)
        
        fifth_tab = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "ul[role='tablist'] li:nth-of-type(5) button")))
        browser.execute_script("arguments[0].scrollIntoView();", fifth_tab)
        browser.execute_script("arguments[0].click();", fifth_tab)
        time.sleep(3)
        match_toss_winner.append(browser.find_element(By.XPATH,"(//div[@class='si-matchinfo-item'])[5]").text)
    time.sleep(3)
    browser.back()
browser.quit()
    